In [1]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import os

#df = pd.read_csv('airline_passenger_satisfaction_cleaned.csv')
# Seleccionar las características (X) y la variable objetivo (y)
#X = df.drop(columns=['satisfaction'])  # Eliminar la columna objetivo del conjunto de características
#y = df['satisfaction']  # Columna objetivo

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear la ruta si no existe
ruta = os.path.abspath('../data/modelos_entrenamiento/')

# Cargar los datos desde los archivos CSV en DataFrames (opcional, si deseas verificar)
X_train = pd.read_csv(os.path.join(ruta, 'X_train.csv'))
y_train = pd.read_csv(os.path.join(ruta, 'y_train.csv'))
X_val_loaded = pd.read_csv(os.path.join(ruta, 'X_val.csv'))
y_val_loaded = pd.read_csv(os.path.join(ruta, 'y_val.csv'))
X_test = pd.read_csv(os.path.join(ruta, 'X_test.csv'))
y_test = pd.read_csv(os.path.join(ruta, 'y_test.csv'))

# Inicializar el modelo CatBoostClassifier
model = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, verbose=100)

# Entrenar el modelo
model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
accuracy = model.score(X_test, y_test)
print(f"Exactitud del modelo en el conjunto de prueba: {accuracy:.2f}")


catboost_train_preds = model.predict(X_train)
catboost_test_preds = model.predict(X_test)
catboost_train_proba = model.predict_proba(X_train)[:, 1]
catboost_test_proba = model.predict_proba(X_test)[:, 1]

catboost_train_recall = recall_score(y_train, catboost_train_preds)
catboost_test_recall = recall_score(y_test, catboost_test_preds)
catboost_train_f1 = f1_score(y_train, catboost_train_preds)
catboost_test_f1 = f1_score(y_test, catboost_test_preds)
catboost_train_auc = roc_auc_score(y_train, catboost_train_proba)
catboost_test_auc = roc_auc_score(y_test, catboost_test_proba)

print(f"CatBoost - Recall en Entrenamiento: {catboost_train_recall:.2f}")
print(f"CatBoost - Recall en Prueba: {catboost_test_recall:.2f}")
print(f"CatBoost - F1 Score en Entrenamiento: {catboost_train_f1:.2f}")
print(f"CatBoost - F1 Score en Prueba: {catboost_test_f1:.2f}")
print(f"CatBoost - AUC en Entrenamiento: {catboost_train_auc:.2f}")
print(f"CatBoost - AUC en Prueba: {catboost_test_auc:.2f}")

0:	learn: 0.5642312	total: 260ms	remaining: 2m 9s
100:	learn: 0.0963266	total: 2.57s	remaining: 10.2s
200:	learn: 0.0807203	total: 5.22s	remaining: 7.77s
300:	learn: 0.0723628	total: 7.88s	remaining: 5.21s
400:	learn: 0.0666081	total: 9.85s	remaining: 2.43s
499:	learn: 0.0624720	total: 11.8s	remaining: 0us
Exactitud del modelo en el conjunto de prueba: 0.96
CatBoost - Recall en Entrenamiento: 0.96
CatBoost - Recall en Prueba: 0.94
CatBoost - F1 Score en Entrenamiento: 0.97
CatBoost - F1 Score en Prueba: 0.96
CatBoost - AUC en Entrenamiento: 1.00
CatBoost - AUC en Prueba: 1.00
